In [2]:
import pandas as pd
from tqdm import tqdm
import json
import os

In [3]:
with open('../Data/BDL/gus_bdl.json', encoding='utf-8') as f:
    data = json.load(f)

In [15]:
def parseData(subject: str, dimension: str, folder: str, out: str):
    records = []
    for key, value in data['subjects'][subject]['dimensions'][dimension]['records'].items():

        for row in value:
            new = {
                'wojewodzctwo': data['territorialUnitNames'][key],
                'rok': row['dateRange']['contents'],
                'wartosc': row['value']
            }
            records.append(new)
    df = pd.DataFrame.from_records(records)
    df.to_csv(f'../Data/BDL/csv/{folder}/{out}.csv', index=False)

In [17]:
for subject in tqdm(data['subjects'].keys()):
    path = data['subjects'][subject]['name'].lower().replace('\n', '__').replace(' ', '_').replace('/', '')
    if not os.path.exists(f'../Data/BDL/csv/{path}'):
        os.makedirs(f'../Data/BDL/csv/{path}')

    for key in data['subjects'][subject]['dimensions'].keys():
        output = key.lower().replace('\n', '__').replace(' ', '_')
        parseData(subject, key, path, output)

100%|██████████| 29/29 [00:00<00:00, 41.35it/s]
